In [ ]:
!pip install -q transformers sentencepiece python-docx pandasimport os
import glob
import pandas as pd
from docx import Document
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import time
from google.colab import drive # Driveマウントのために追加

# --- モデルとトークナイザーの準備 (流用) ---
model_name = "tsmatz/mt5_summarize_japanese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# GPUが利用可能であれば使用
if torch.cuda.is_available():
    model.to('cuda')

def summarize(text: str, target_chars=120):
    """指定された文字数でテキストを要約する関数"""
    max_length = int(target_chars * 1.3)

    prompt = (
        f"次の文章を、だいたい {target_chars} 文字くらいの長さで自然に要約してください。\n"
        f"【文章】{text}\n"
        "【要約】"
    )

    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

    if torch.cuda.is_available():
        inputs = inputs.to('cuda')

    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def read_docx(file_path):
    """Wordファイル (.docx) からテキストを抽出する関数"""
    try:
        document = Document(file_path)
        full_text = "\n".join([p.text.strip() for p in document.paragraphs if p.text.strip()])
        return full_text
    except Exception as e:
        print(f"  🚨 Wordファイル読み込みエラー {file_path}: {e}")
        return None

def summarize_folder_to_csv(folder_path: str, target_chars: int, output_csv: str):
    """フォルダ内のすべての.docxファイルを要約し、結果をCSVに保存するメイン関数"""

    # フォルダが存在するかチェック
    if not os.path.isdir(folder_path):
        print(f"🛑 エラー: 指定されたフォルダが見つかりません。パスを確認してください: {folder_path}")
        return

    print(f"--- フォルダ内の.docxファイルを検索中: {folder_path} ---")

    file_list = glob.glob(os.path.join(folder_path, "*.docx"))

    if not file_list:
        print("🛑 指定されたフォルダ内に.docxファイルが見つかりませんでした。")
        return

    results = []
    print(f"--- ✅ {len(file_list)}個のファイルを見つけました。処理を開始します ---")

    for i, file_path in enumerate(file_list):
        start_time = time.time()
        file_name = os.path.basename(file_path)
        print(f"\n[{i+1}/{len(file_list)}] 📄 処理中: {file_name}")

        original_text = read_docx(file_path)

        if original_text and original_text.strip():
            try:
                summary_text = summarize(original_text, target_chars=target_chars)
                actual_length = len(summary_text)
                elapsed_time = time.time() - start_time
                print(f"   -> 要約完了 (文字数: {actual_length}, 処理時間: {elapsed_time:.2f}秒)")

                results.append({
                    "ファイル名": file_name,
                    "元のテキスト（先頭100文字）": original_text[:100].replace('\n', ' ') + "...",
                    "要約文字数_希望": target_chars,
                    "要約文字数_実際": actual_length,
                    "要約結果": summary_text
                })
            except Exception as e:
                print(f"   🚨 要約中にエラーが発生しました: {e}")
                results.append({
                    "ファイル名": file_name,
                    "元のテキスト（先頭100文字）": "処理エラー",
                    "要約文字数_希望": target_chars,
                    "要約文字数_実際": 0,
                    "要約結果": f"要約エラー: {e}"
                })
        else:
            print("   ⚠️ テキストが空か、読み込めませんでした。スキップします。")
            results.append({
                "ファイル名": file_name,
                "元のテキスト（先頭100文字）": "テキストなし/読み込み失敗",
                "要約文字数_希望": target_chars,
                "要約文字数_実際": 0,
                "要約結果": "処理スキップ"
            })

    # 結果をPandas DataFrameにしてCSVファイルとして保存
    if results:
        df = pd.DataFrame(results)
        df.to_csv(output_csv, index=False, encoding='utf-8-sig')
        print(f"\n--- ✅ すべての処理が完了しました。結果をCSVに保存しました: {output_csv} ---")
    else:
        print("\n--- 処理対象ファイルがないため、CSVファイルは作成されませんでした。 ---")


# ==================================
# === 実行部分 (Google Colab / パス固定) ===
# ==================================

print("--- 📂 Google Driveをマウントします ---")
# 認証プロセスを実行し、Google Driveを '/content/drive' にマウント
drive.mount('/content/drive')

# ユーザーの指定に基づき、フォルダパスを固定
FOLDER_NAME = "tv"
folder_path = f"/content/drive/MyDrive/{FOLDER_NAME}"

print(f"\n--- 🎯 ターゲットフォルダ: {folder_path} ---")

# 目標文字数はユーザーに入力してもらう
try:
    target = int(input("だいたい何文字くらいで要約してほしいですか？："))
except ValueError:
    print("入力が不正です。デフォルトの120文字を使用します。")
    target = 120

# 出力ファイル名を設定
# 結果のCSVファイルは、読み込み元の newspaper フォルダ内に保存されます。
timestamp = time.strftime('%Y%m%d_%H%M%S')
output_file = os.path.join(folder_path, f"summary_results_{timestamp}.csv")

# メイン関数を実行
summarize_folder_to_csv(folder_path, target_chars=target, output_csv=output_file)